# **Packages**

In [102]:
import pandas as pd
import numpy as np
import pickle

from stores_module import *
from recursive_predictions import *
from sales_module import *

# **Datasets**

In [103]:
with open("AngeliqueFile.pkl", "rb") as f:
    transactions = pickle.load(f)

In [104]:
train_set = pd.read_csv("train.csv")
train_set["date"] = pd.to_datetime(train_set["date"])

In [105]:
with open("transac_sales.pkl","rb") as f:
    transac_sales = pickle.load(f)

# **Prediction transactions**

In [106]:
stores_list = list(transactions["store_nbr"].unique())


In [107]:
stores_test = transactions.loc[transactions["date"] >= "2017-08-01","date"].to_frame()
stores_test.drop_duplicates(inplace=True, ignore_index=True)

In [108]:
for store in stores_list:
    store_df = store_data(store)
    no_cat_list = ["not a national event","Not a Nat holiday","Nope"]
    for col in store_df.columns:
        for cat in no_cat_list:
            if store_df[col].dtype == "category" and cat in store_df[col].cat.categories:
                store_df[col] = rename_null_cat(store_df,col,cat)
        if store_df[col].dtype == "category" and len(store_df[col].cat.categories) == 2:
            col_binom = binom_cat_bool(store_df)
            col_binom.binom_bool()
            store_df.loc[:,col] = col_binom.transform()

        elif store_df[col].dtype == "category" and len(store_df[col].cat.categories) > 2:
            col_Encod = my_labelEncoder()
            col_Encod.fit(store_df,col)
            store_df[col] = col_Encod.transform(store_df,col)
    
    _,store_df = frame_time_of_interest(store_df,8) #To generalise we need to precise the month upstream
    split_Lagg = SplitLagg(store_df)
    split_Lagg.transactions_X()
    X_df = split_Lagg.lagg_X(lags=4)
    y_df = split_Lagg.lagg_y(lags=4)
    X_train,y_train,X_test,y_test,test_y = split_train_test(X_df,y_df,date="2017-07-31")
    predictions_store, test_yPred = recurs_Lin_regr(X_train,y_train,X_test,test_y)
    if "store_nbr" not in stores_test.columns:
        stores_test["store_nbr"] = [25] * len(y_test)
        stores_test["value"] = y_test.values
        stores_test["transactions_pred"] = predictions_store
    else:
        store_dict = {"date": y_test.index, "store_nbr": [store] * len(y_test), "value": y_test.values, "transactions_pred": predictions_store}
        store_dict = pd.DataFrame(store_dict)
        stores_test = pd.concat([stores_test, store_dict], ignore_index=True, axis=0)
    stores_test["transactions_pred"] = stores_test["transactions_pred"].astype("int")

In [109]:
stores_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 810 entries, 0 to 809
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               810 non-null    datetime64[ns]
 1   store_nbr          810 non-null    int64         
 2   value              810 non-null    int64         
 3   transactions_pred  810 non-null    int32         
dtypes: datetime64[ns](1), int32(1), int64(2)
memory usage: 22.3 KB


In [110]:
stores_test_52 = stores_test.loc[stores_test["store_nbr"] == 52]
stores_test_52

,date,store_nbr,value,transactions_pred
795,2017-08-01,52,2147,1905
796,2017-08-02,52,2499,1975
797,2017-08-03,52,2045,1783
798,2017-08-04,52,2442,2125
799,2017-08-05,52,2837,2765
800,2017-08-06,52,2711,2599
801,2017-08-07,52,2152,1829
802,2017-08-08,52,1997,1644
803,2017-08-09,52,2300,1820
804,2017-08-10,52,2165,3621


# **Preprocessing**

In [111]:
train = train_set[train_set["date"] <"2017-08-01"].copy()
test = train_set[train_set["date"] >= "2017-08-01"].copy()

In [112]:
transactions["date"].min()

Timestamp('2013-01-01 00:00:00')

In [113]:
transactions_train = transactions[transactions["date"] < "2017-08-01"].copy()
train = train.merge(transactions_train[["date","transactions","store_nbr"]], \
                    on=["date","store_nbr"], how="left", validate="many_to_one")

In [114]:
train.isna().sum()

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
transactions    245784
dtype: int64

In [115]:
train.fillna(0, inplace=True)

In [116]:
test = test.merge(stores_test[["date","store_nbr", "transactions_pred"]], \
                  on=["date","store_nbr"], how="left", validate="many_to_one")

In [117]:
test["transactions"] = test["transactions_pred"]
test.drop("transactions_pred", axis=1, inplace=True)

In [118]:
test.isna().sum()

id              0
date            0
store_nbr       0
family          0
sales           0
onpromotion     0
transactions    0
dtype: int64

In [119]:
train.isna().sum()

id              0
date            0
store_nbr       0
family          0
sales           0
onpromotion     0
transactions    0
dtype: int64

In [120]:
sales = pd.concat([train,test], axis=0)

In [121]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000888 entries, 0 to 26729
Data columns (total 7 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            int64         
 1   date          datetime64[ns]
 2   store_nbr     int64         
 3   family        object        
 4   sales         float64       
 5   onpromotion   int64         
 6   transactions  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 183.2+ MB


## **Adding events and holidays**

In [122]:
holid_transac = transactions_cat()

In [123]:
holid_transac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date                        83488 non-null  datetime64[ns]
 1   store_nbr                   83488 non-null  int64         
 2   transactions                83488 non-null  int64         
 3   National Event              83488 non-null  Int64         
 4   National holiday            83488 non-null  Int64         
 5   National period of holiday  83488 non-null  Int64         
 6   National Workday            83488 non-null  int32         
 7   city                        83488 non-null  Int64         
 8   state                       83488 non-null  Int64         
 9   type                        83488 non-null  Int64         
 10  cluster                     83488 non-null  int64         
 11  Local Holiday               83488 non-null  int32     

In [124]:
sales["date"].min()

Timestamp('2013-01-01 00:00:00')

In [125]:
sales = sales.merge(holid_transac[["date","store_nbr", "payday","Local Holiday","Regional Holiday","Workday","National Workday", "National Event","National holiday",\
                                   "National period of holiday","Transfer"]],\
                                    on=["date","store_nbr"], how="left", validate="many_to_one")

In [126]:
sales.isna().sum()

id                                 0
date                               0
store_nbr                          0
family                             0
sales                              0
onpromotion                        0
transactions                       0
payday                        245784
Local Holiday                 245784
Regional Holiday              245784
Workday                       245784
National Workday              245784
National Event                245784
National holiday              245784
National period of holiday    245784
Transfer                      245784
dtype: int64

In [127]:
missing_vals = sales[sales["Local Holiday"].isna()].copy()
dates_missing = list(missing_vals["date"].unique())

In [128]:
missing_vals["transactions"].describe()

count    245784.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: transactions, dtype: float64

In [129]:
cols_to_fill = ["payday","Local Holiday","Regional Holiday","Workday","National Workday","National Event","National holiday",\
                                                                        "National period of holiday","Transfer"]
for date in dates_missing:
    ref_row = sales[(sales["date"] == date)&(sales["transactions"] != 0)]
    if not ref_row.empty:
        ref_values = ref_row.iloc[0][cols_to_fill]
        sales.loc[(sales["date"] == date)&(sales["transactions"] == 0), cols_to_fill] = ref_values.values

In [130]:
sales.isna().sum()

id                               0
date                             0
store_nbr                        0
family                           0
sales                            0
onpromotion                      0
transactions                     0
payday                        3564
Local Holiday                 3564
Regional Holiday              3564
Workday                       3564
National Workday              3564
National Event                3564
National holiday              3564
National period of holiday    3564
Transfer                      3564
dtype: int64

In [131]:
sales.loc[sales["Local Holiday"].isna(),"date"].unique()

<DatetimeArray>
['2016-01-01 00:00:00', '2016-01-03 00:00:00']
Length: 2, dtype: datetime64[ns]

In [132]:
sales.loc[sales["Local Holiday"].isna(),"transactions"].describe()

count    3564.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: transactions, dtype: float64

In [133]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 16 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   id                          int64         
 1   date                        datetime64[ns]
 2   store_nbr                   int64         
 3   family                      object        
 4   sales                       float64       
 5   onpromotion                 int64         
 6   transactions                float64       
 7   payday                      float64       
 8   Local Holiday               float64       
 9   Regional Holiday            float64       
 10  Workday                     float64       
 11  National Workday            float64       
 12  National Event              Int64         
 13  National holiday            Int64         
 14  National period of holiday  Int64         
 15  Transfer                    float64       
dtypes: Int64(3), datet

In [134]:
sales.drop(sales.loc[sales["date"].isin(["2016-01-01","2016-01-03"])].index, axis=0, inplace=True)

C:\Users\Angelique\AppData\Local\Temp\ipykernel_5744\1626311016.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  sales.drop(sales.loc[sales["date"].isin(["2016-01-01","2016-01-03"])].index, axis=0, inplace=True)


In [135]:
sales.isna().sum()

id                            0
date                          0
store_nbr                     0
family                        0
sales                         0
onpromotion                   0
transactions                  0
payday                        0
Local Holiday                 0
Regional Holiday              0
Workday                       0
National Workday              0
National Event                0
National holiday              0
National period of holiday    0
Transfer                      0
dtype: int64

In [136]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2997324 entries, 0 to 3000887
Data columns (total 16 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   id                          int64         
 1   date                        datetime64[ns]
 2   store_nbr                   int64         
 3   family                      object        
 4   sales                       float64       
 5   onpromotion                 int64         
 6   transactions                float64       
 7   payday                      float64       
 8   Local Holiday               float64       
 9   Regional Holiday            float64       
 10  Workday                     float64       
 11  National Workday            float64       
 12  National Event              Int64         
 13  National holiday            Int64         
 14  National period of holiday  Int64         
 15  Transfer                    float64       
dtypes: Int64(3), datetime64

In [137]:
sales.isna().sum()

id                            0
date                          0
store_nbr                     0
family                        0
sales                         0
onpromotion                   0
transactions                  0
payday                        0
Local Holiday                 0
Regional Holiday              0
Workday                       0
National Workday              0
National Event                0
National holiday              0
National period of holiday    0
Transfer                      0
dtype: int64

In [138]:
sales.loc[(sales["family"] == "AUTOMOTIVE")&(sales["date"] >= "2017-08-01")&(sales["store_nbr"] == 52)]

,id,date,store_nbr,family,sales,onpromotion,transactions,payday,Local Holiday,Regional Holiday,Workday,National Workday,National Event,National holiday,National period of holiday,Transfer
2975709,2975709,2017-08-01,52,AUTOMOTIVE,5.0,0,1905.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0
2977491,2977491,2017-08-02,52,AUTOMOTIVE,19.0,0,1975.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0
2979273,2979273,2017-08-03,52,AUTOMOTIVE,19.0,0,1783.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0
2981055,2981055,2017-08-04,52,AUTOMOTIVE,9.0,0,2125.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0
2982837,2982837,2017-08-05,52,AUTOMOTIVE,15.0,0,2765.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0
2984619,2984619,2017-08-06,52,AUTOMOTIVE,24.0,0,2599.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0
2986401,2986401,2017-08-07,52,AUTOMOTIVE,5.0,0,1829.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0
2988183,2988183,2017-08-08,52,AUTOMOTIVE,8.0,0,1644.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0
2989965,2989965,2017-08-09,52,AUTOMOTIVE,11.0,0,1820.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0
2991747,2991747,2017-08-10,52,AUTOMOTIVE,7.0,0,3621.0,0.0,0.0,0.0,0.0,0.0,0,7,0,0.0


# **Predictions looping through families and stores**

In [139]:
items_list = list(sales["family"].unique())

In [140]:
items_list

['AUTOMOTIVE',
 'BABY CARE',
 'BEAUTY',
 'BEVERAGES',
 'BOOKS',
 'BREAD/BAKERY',
 'CELEBRATION',
 'CLEANING',
 'DAIRY',
 'DELI',
 'EGGS',
 'FROZEN FOODS',
 'GROCERY I',
 'GROCERY II',
 'HARDWARE',
 'HOME AND KITCHEN I',
 'HOME AND KITCHEN II',
 'HOME APPLIANCES',
 'HOME CARE',
 'LADIESWEAR',
 'LAWN AND GARDEN',
 'LINGERIE',
 'LIQUOR,WINE,BEER',
 'MAGAZINES',
 'MEATS',
 'PERSONAL CARE',
 'PET SUPPLIES',
 'PLAYERS AND ELECTRONICS',
 'POULTRY',
 'PREPARED FOODS',
 'PRODUCE',
 'SCHOOL AND OFFICE SUPPLIES',
 'SEAFOOD']

In [141]:
sales.loc[sales["family"] == "BREAD/BAKERY","family"] = "BREAD"

In [142]:
items_list = list(sales["family"].unique())
items_list

['AUTOMOTIVE',
 'BABY CARE',
 'BEAUTY',
 'BEVERAGES',
 'BOOKS',
 'BREAD',
 'CELEBRATION',
 'CLEANING',
 'DAIRY',
 'DELI',
 'EGGS',
 'FROZEN FOODS',
 'GROCERY I',
 'GROCERY II',
 'HARDWARE',
 'HOME AND KITCHEN I',
 'HOME AND KITCHEN II',
 'HOME APPLIANCES',
 'HOME CARE',
 'LADIESWEAR',
 'LAWN AND GARDEN',
 'LINGERIE',
 'LIQUOR,WINE,BEER',
 'MAGAZINES',
 'MEATS',
 'PERSONAL CARE',
 'PET SUPPLIES',
 'PLAYERS AND ELECTRONICS',
 'POULTRY',
 'PREPARED FOODS',
 'PRODUCE',
 'SCHOOL AND OFFICE SUPPLIES',
 'SEAFOOD']

In [143]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2997324 entries, 0 to 3000887
Data columns (total 16 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   id                          int64         
 1   date                        datetime64[ns]
 2   store_nbr                   int64         
 3   family                      object        
 4   sales                       float64       
 5   onpromotion                 int64         
 6   transactions                float64       
 7   payday                      float64       
 8   Local Holiday               float64       
 9   Regional Holiday            float64       
 10  Workday                     float64       
 11  National Workday            float64       
 12  National Event              Int64         
 13  National holiday            Int64         
 14  National period of holiday  Int64         
 15  Transfer                    float64       
dtypes: Int64(3), datetime64

In [144]:
print(sales.isna().sum())

id                            0
date                          0
store_nbr                     0
family                        0
sales                         0
onpromotion                   0
transactions                  0
payday                        0
Local Holiday                 0
Regional Holiday              0
Workday                       0
National Workday              0
National Event                0
National holiday              0
National period of holiday    0
Transfer                      0
dtype: int64


In [145]:
sales.loc

In [146]:
stores_list[0]

25

In [147]:
AUTOMOTIVE_df = family_df(family_name="AUTOMOTIVE",df=sales, month=8)

In [148]:
auto_52 = AUTOMOTIVE_df.loc[AUTOMOTIVE_df["store_nbr"] == 52].copy()
auto_52.drop("store_nbr", axis=1, inplace=True)
to_be_lagged = auto_52[["sales","transactions","weekday"]].copy()
lagg = SplitLagg_sale(to_be_lagged,"sales")
lagg.sales_X()
X_lagged = lagg.lagg_X(4)
y_lagged = lagg.lagg_y(4)
cols_to_remove = list(to_be_lagged.columns)
auto_52.drop(cols_to_remove, axis=1, inplace=True)
auto_52 = pd.concat([auto_52,X_lagged,y_lagged], axis=1)
X = auto_52.drop("var1 y(t)", axis=1)
y = auto_52[["var1 y(t)"]]
X_train = X.loc[:"2017-07-31"]
y_train = y.loc[:"2017-07-31"]
X_test = X.loc["2017-08-01":]
y_test = y.loc["2017-08-01":]
y_test

,var1 y(t)
date,
2017-08-01,5.0
2017-08-02,19.0
2017-08-03,19.0
2017-08-04,9.0
2017-08-05,15.0
2017-08-06,24.0
2017-08-07,5.0
2017-08-08,8.0
2017-08-09,11.0


In [149]:
aug_dates_25 = list(sales.loc[(sales["store_nbr"] == stores_list[0])&(sales["date"] >= "2017-08-01"),"date"].unique())

In [151]:
for item in items_list:
    item_df = family_df(family_name=item,df=sales,month=8)
    stores_pred_df = pd.DataFrame(aug_dates_25, columns=["date"])
    for store in stores_list:
        store_df = item_df[item_df["store_nbr"] == store].copy()
        store_df.drop("store_nbr", axis=1, inplace=True)
        to_be_lagged = store_df[["sales","transactions","weekday"]].copy()
        lagg = SplitLagg_sale(to_be_lagged,"sales")
        lagg.sales_X()
        lagg_X = lagg.lagg_X(4)
        lagg_y = lagg.lagg_y(4)
        cols_to_remove = list(to_be_lagged.columns)
        store_df = store_df.drop(cols_to_remove,axis=1)
        store_df = store_df.iloc[4:]
        store_df = pd.concat([store_df,lagg_X,lagg_y],axis=1)
        #split
        X = store_df.drop("var1 y(t)", axis=1)
        y = store_df[["var1 y(t)"]]
        X_train = X.loc[:"2017-07-31"]
        y_train = y.loc[:"2017-07-31"]
        X_test = X.loc["2017-08-01":]
        y_test = y.loc["2017-08-01":]
        index=len(X.columns) - 4 #index of the col from which we add the pred laggs
        test_y = X_test.iloc[:,index:].copy()
        if len(X_test)==0:
            print(f"The store {store} doesn't have any {item}")
            break
        elif len(X_test) < 15:
            print(f"Store {store} has missing dates for item {item}")
        predictions, test_pred = recurs_Lin_regr(X_train,y_train,X_test,test_y)
        if store == stores_list[0]:
            stores_pred_df["store_nbr"] = [store] * len(y_test)
            stores_pred_df["value"] = y_test.values
            stores_pred_df["prediction"] = predictions
        else:
            store_dict_i = {"date":X_test.index, "store_nbr":[store]*len(y_test), "value":y_test["var1 y(t)"],\
                          "prediction":predictions}
            store_dict_i = pd.DataFrame(store_dict_i)
            stores_pred_df = pd.concat([stores_pred_df, store_dict_i], ignore_index=True, axis=0)
            stores_pred_df["prediction"] = stores_pred_df["prediction"].astype("int")
    with open(f"{item}_pred.pkl", "wb") as f:
        pickle.dump(stores_pred_df,f)
        